In [83]:
import itertools
import numpy as np
import pandas as pd

In [2]:
def getcolumn(M,i):
    ret = np.array([])
    for linha in M:
        ret = np.append(ret,[linha[i]])
    return ret

In [87]:
class Solve:
    
    def __init__(self,A,b,c) -> None:
        # formato padrão
        self.A = np.array(A)
        self.b = np.array(b)
        self.c = np.array(c)
        self.n = len(c)
        self.m = len(b)


    def matrixBase(self, indices):
        B = []
        for i in range(len(self.A)):
            linha = []
            for j in range(len(self.A[0])):
                if j in indices:
                    linha.append(self.A[i,j])
            B.append(linha)
        return np.array(B,ndmin=2)


    def solveBase(self,indices):
        B = self.matrixBase(indices)
        invB = np.linalg.inv(B)
        xb = iter(np.dot(invB,self.b))
        x = []
        for i in range(self.n):
            if i in indices:
                x.append(next(xb))
            else:
                x.append(0)
        return x


    def cost(self,x):
        return np.dot(np.transpose(self.c),x)


    def viable(self,x):
        for x_ in x: 
            if x_ < 0: 
                return False
        return True 
    

    def degenerated(self,x):
        return list(x).count(0) > self.n - self.m


    def tryAll(self):
        LB,Lx,Lctx,Lviable,Ldeg = [],[],[],[],[]        
        for indices in itertools.combinations(list(range(self.n)),self.m):
            x = self.solveBase(indices)
            LB.append(self.matrixBase(indices))
            Lx.append(x)
            Lctx.append(self.cost(x))
            Lviable.append(self.viable(x))
            Ldeg.append(self.degenerated(x))

        df = pd.DataFrame({
            'B':LB,
            'x':Lx,
            'custo':Lctx,
            'viavel':Lviable,
            'degenerada': Ldeg
        })
        return df


In [89]:
s = Solve([[1,2,3,4],[3,4,5,6]],[1,2],[9,-8,1,2])
s.tryAll()

,B,x,custo,viavel,degenerada
0,"[[1, 2], [3, 4]]","[0.0, 0.5, 0, 0]",-4.000000e+00,True,True
1,"[[1, 3], [3, 5]]","[0.2500000000000002, 0, 0.25, 0]",2.500000e+00,True,False
2,"[[1, 4], [3, 6]]","[0.33333333333333326, 0, 0, 0.16666666666666669]",3.333333e+00,True,False
3,"[[2, 3], [4, 5]]","[0, 0.5, 0.0, 0]",-4.000000e+00,True,True
4,"[[2, 4], [4, 6]]","[0, 0.5, 0, 0.0]",-4.000000e+00,True,True
5,"[[3, 4], [5, 6]]","[0, 0, 1.0000000000000018, -0.5000000000000013]",-8.881784e-16,False,False
